In [0]:

import datetime
start = "2016-08-28"
end = "2020-03-01"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(str(dates.pop()))
        sar_list.append(str(real_date1 + datetime.timedelta(days)))

for x in sar_list:
    print x





In [0]:

import datetime
start = "2016-08-28"
end = "2020-03-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append((str(real_date1 + datetime.timedelta(days))))
dates.sort(reverse=True)


In [0]:
%%sh

# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3







# aws s3 ls s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/2020-04-04/AU/
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v4/fact/granularity=daily/date=
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/download-attribution.v1/fact/month=2020-07/


In [0]:

spark.read.parquet("s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/2020-06-13/US/").where("product_id=284882215").show()
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/download-attribution.v1/fact/month=2020-06/date=2020-06-13/device_id=2001/store_id=143441/").where("app_id=284882215").show()

# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v4/fact/granularity=daily/date=2020-07-02/").where("country_code='US' and device_code='ios-phone'").show()


In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
select app_id, store_id, device_id, est_organic_download_share from plproxy.execute_select_nestloop(\$proxy\$ 
select distinct app_id,store_id, device_id, est_organic_download_share
    from da.app_da_daily_estimate_2001 where date = '2020-03-28' and app_id = 284882215 and store_id = 143441
     limit 5 \$proxy\$) tbl 
      (app_id BIGINT, store_id INT, device_id SMALLINT, est_organic_download_share FLOAT )  ;

EOF


In [0]:
%%sh
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-dna-log.v1/fact/granularity=daily/date=2020-01-01/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log-pre-aggr.v1/fact/granularity=weekly/ --recursive
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-category-load.v3/fact/granularity=daily/


In [0]:

spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-category-load.v3/fact/").createOrReplaceTempView("temp")
spark.sql('''
            SELECT Sum(est_paid_download)                                  AS total_est_paid_download
        FROM   (
                        SELECT   Max(est_paid_download) AS est_paid_download,
                                 device_code,
                                 country_code,
                                 date,
                                 app_id
                        FROM     temp
                        WHERE    granularity = 'daily'
                        AND      date between '2020-08-02' and '2020-08-03'
                        GROUP BY device_code,
                                 country_code,
                                 date,
                                 app_id) AS prod

    
   ''' ).show()


In [0]:

spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-dna-log-pre-aggr-pubrange.v1/fact/").createOrReplaceTempView("temp")
spark.sql('''
            SELECT Sum(est_paid_download)                                  AS total_est_paid_download,
                    Sum(est_free_app_download)                                  AS est_free_app_download,
                    Sum(est_paid_app_download)                                  AS est_paid_app_download,
                    Sum(est_revenue)                                  AS est_revenue,
                    Sum(est_organic_download)                                  AS est_organic_download
            
        FROM   (
                        SELECT   Max(est_paid_download) AS est_paid_download,
                                Max(est_free_app_download) AS est_free_app_download,
                                 Max(est_paid_app_download) AS est_paid_app_download,
                                Max(est_revenue) AS est_revenue,
                                 Max(est_organic_download) AS est_organic_download,

                                 device_code,
                                 country_code,
                                 date,
                                 app_id
                        FROM     temp
                        WHERE    granularity = 'weekly'
                        AND      month = '2019-10'
                        AND       pub_range=999
                        AND       device_code='ios-phone'
                        GROUP BY device_code,
                                 country_code,
                                 date,
                                 app_id) AS prod

    
   ''' ).show()



spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-dna-log-pre-aggr-pubrange.v2/fact/").createOrReplaceTempView("temp_2")
spark.sql('''
            SELECT Sum(est_paid_download)                                  AS total_est_paid_download,
                    Sum(est_free_app_download)                                  AS est_free_app_download,
                    Sum(est_paid_app_download)                                  AS est_paid_app_download,
                    Sum(est_revenue)                                  AS est_revenue,
                    Sum(est_organic_download)                                  AS est_organic_download
        FROM   (
                        SELECT   Max(est_paid_download) AS est_paid_download,
                                Max(est_free_app_download) AS est_free_app_download,
                                 Max(est_paid_app_download) AS est_paid_app_download,
                                Max(est_revenue) AS est_revenue,
                                 Max(est_organic_download) AS est_organic_download,
                                 device_code,
                                 country_code,
                                 date,
                                 app_id
                        FROM     temp_2
                        WHERE    granularity = 'weekly'
                        AND      month = '2019-10'
                        AND       pub_range=999
                        AND       device_code='ios-phone'
                        GROUP BY device_code,
                                 country_code,
                                 date,
                                 app_id) AS prod

    
   ''' ).show()


In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log-pre-aggr.v1/fact/granularity=weekly/
 
 

In [0]:

spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log-pre-aggr.v1/fact/").createOrReplaceTempView("temp_agg")
spark.sql("select * from temp_agg where granularity='weekly' and date between '2019-09-28' and '2019-11-02' and app_id = 1459475003 and date='2019-10-05' and country_code='KR' order by date asc").show()




In [0]:

spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-dna-log-pre-aggr-pubrange.v1/fact/").createOrReplaceTempView("temp_1")
spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-dna-log-pre-aggr-pubrange.v2/fact/").createOrReplaceTempView("temp_2")

print spark.sql("select distinct app_id from temp_1 WHERE granularity = 'weekly' AND month = '2019-10' except all select distinct app_id from temp_2 WHERE granularity = 'weekly'  AND  month = '2019-10'").collect()
print spark.sql("select distinct app_id from temp_2 WHERE granularity = 'weekly' AND month = '2019-10' except all select distinct app_id from temp_1 WHERE granularity = 'weekly'  AND  month = '2019-10'").collect()

# spark.sql(''' select * from temp_1   WHERE granularity = 'weekly'
#                         AND      month = '2019-10'
#                         AND       pub_range=999
#                         AND       device_code='ios-phone'
#                         AND app_id = 1440025446
                        
#  ''').show()
 
 
 
# spark.sql(''' select * from temp_2   WHERE    granularity = 'weekly'
#                         AND      month = '2019-10'
#                         AND       pub_range=999
#                         AND       device_code='ios-phone'
#                         AND app_id = 1440025446

#  ''').show()

In [0]:

# 1459475003, 20600011643145, 20600012536233, 1300096411, 20600012556110, 1457824642, 1306783602, 1476812245
spark.sql("select * from temp_2 where granularity = 'weekly'  AND  month = '2019-10' and app_id in (1459475003 )  ").show()

In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log-update.v1/fact/granularity=weekly/year=2019/_update_date=2020-07-07/").createOrReplaceTempView("pub")
spark.sql("select distinct app_id , publisher_id_mod from ( select *, mod(publisher_id, 1000) AS publisher_id_mod from pub ) ").show()

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-dna-log-pubrange.v1/fact/granularity=daily/month=2020-08/pub_range=252/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-dna-log-pre-aggr-pubrange.v1/fact/granularity=weekly/month=2019-10/pub_range=103/device_code=ios-phone/
 
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-dna-log-pre-aggr-pubrange.v2/fact/granularity=weekly/month=2019-10/pub_range=103/device_code=ios-phone/

 

# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/date=2020-05


In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log-update.v1/fact/granularity=daily/year=2019/_update_date=2020-07-07/

 
 


In [0]:



df_daily = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/date=2020-05-{24,25,26,27,28,29,30}").cache()
# df_daily = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/date=2020-03*/").cache()
df_daily.createOrReplaceTempView("dr_daily")
df_weekly = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log-pre-aggr.v1/fact/granularity=weekly/date=2020-05-30/").cache()
df_weekly.createOrReplaceTempView("dr_weekly")

spark.sql("select app_id, country_code, device_code, publisher_id, company_id, parent_company_id , sum(est_free_app_download) as est_free_app_download, sum(est_paid_app_download) as est_paid_app_download, sum(est_revenue) as est_revenue, sum(est_paid_download) as est_paid_download, sum(est_organic_download) as est_organic_download from dr_daily group by app_id, country_code, device_code,  publisher_id, company_id, parent_company_id except select app_id, country_code, device_code, publisher_id, company_id, parent_company_id, est_free_app_download, est_paid_app_download, est_revenue, est_paid_download, est_organic_download  from dr_weekly").show()
spark.sql("select app_id, country_code, device_code, publisher_id, company_id, parent_company_id, est_free_app_download, est_paid_app_download, est_revenue, est_paid_download, est_organic_download from dr_weekly except select app_id, country_code, device_code, publisher_id, company_id, parent_company_id, sum(est_free_app_download) as est_free_app_download, sum(est_paid_app_download) as est_paid_app_download, sum(est_revenue) as est_revenue, sum(est_paid_download) as est_paid_download, sum(est_organic_download) as est_organic_download from dr_daily group by app_id, country_code, device_code,  publisher_id, company_id, parent_company_id").show()


In [0]:



daily_est = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/date=2020-02*/").cache()
daily_est.createOrReplaceTempView("daily_est")
# spark.sql("select count(1), sum(est_paid_download), sum(est_organic_download), sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue) from daily_est").show()


# daily = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-dna-log.v1/fact/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-dna-log.v1/fact/granularity=daily/date=2020-03*/").cache()
# daily.createOrReplaceTempView("daily")

# spark.sql("select count(1), sum(est_paid_download), sum(est_organic_download), sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue) from daily").show()


# daily_download_attr_preload = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/").where("granularity='monthly' and date between '2020-03-01' and '2020-03-31'").cache()
# daily_download_attr_preload.createOrReplaceTempView("daily_download_attr_preload")

# spark.sql("select count(1), sum(est_paid_download), sum(est_organic_download), sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue) from daily_download_attr_preload").show()


# monthly = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-dna-log-pre-aggr.v1/fact/granularity=monthly/date=2020-02-29/").cache()
# monthly.createOrReplaceTempView("pre_agg")
# spark.sql("select count(1), sum(est_paid_download), sum(est_organic_download), sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue) from pre_agg ").show()
# 20600012931082
# 20600010012490

dd = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log-pre-aggr.v1/fact/granularity=monthly/date=2020-02-29").cache()
dd.createOrReplaceTempView("pre_agg_est")
spark.sql("select * from  pre_agg_est  where app_id = 20600012931082 and country_code='US'").show()
spark.sql("select * from  pre_agg  where app_id = 20600012931082 and country_code='US' ").show()
spark.sql("select * from daily_est where app_id = 20600012931082 and country_code='US' order by date desc").show(2000)

+------------------+--------------+----------------+----------+-----------------+--------------+----------+
|       _identifier|        app_id|      company_id|  end_date|parent_company_id|  publisher_id|start_date|
+------------------+--------------+----------------+----------+-----------------+--------------+----------+
|120200630134627330|20600012931082|1000200000103558|2015-09-15|                0|20200001825692|1970-01-01|
|120200630134627330|20600012931082|1000200000103558|2020-02-07|                0|20200001825692|2015-09-16|
|120200630134627330|20600012931082|               0|2020-02-15|                0|20200003444285|2020-02-08|
|120200630134627330|20600012931082|               0|2020-02-25|                0|20200003452891|2020-02-16|
|120200630134627330|20600012931082|1000200000449440|2020-03-18| 1000200000044132|20200003465209|2020-02-26|
|120200630134627330|20600012931082|1000200000449440|      null| 1000200000044132|20200003490885|2020-03-19|
+------------------+--------------+----------------+----------+-----------------+--------------+----------+


In [0]:

spark.sql("select count (distinct app_id) from daily_est").show()

spark.sql("select count (distinct app_id) from daily").show()
spark.sql("select * from pre_agg").show()

In [0]:

spark.sql("select app_id, count(app_id) as count_app, publisher_id, company_id from pre_agg where country_code='AU' and device_code='ios-phone' group by app_id, publisher_id, company_id, parent_company_id, device_code order by count_app desc ").show()

In [0]:

spark.sql("select sum(est_paid_download), sum(est_organic_download), sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue)  from daily_est where app_id = 284862083 and country_code='US' and device_code='ios-phone'  ").show(50)


spark.sql("select * from daily where app_id = 284862083 and country_code='AU' and device_code='ios-phone' order by date ").show()
spark.sql("select * from pre_agg where app_id = 284862083 and country_code='AU' and device_code='ios-phone'  ").show()

In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-dna-log.v1/fact/granularity=daily/date=2020-03-01/").createOrReplaceTempView("temp")
# spark.sql("select count(*) from temp").show()
spark.sql("select * from temp where app_id = 20600012931082 and country_code in ( 'US') ").show()

spark.sql("select * from daily_est where app_id = 20600012931082 and country_code in ('US')  ").show()


In [0]:

from pyspark.sql import functions as F
df_dna = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/update_date=2020-06-30/").cache()
df_event = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-publisher-event.v1/dimension/update_date=2020-06-30/").cache()

df_dna.createOrReplaceTempView("dna_logging")
spark.sql("select * from dna_logging where app_id=20600012931082 ").show(10)


In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F


date='2020-02-23'

def plproxy_row(date):
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )

    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': host,
            'PORT': port,
            'NAME': getattr(settings, template.format(property='NAME')),
            'PASSWORD': getattr(settings, template.format(property='SECRET_KEY')),
            'USER': getattr(settings, template.format(property='ACCESS_ID')),
            'NODE_NUM': int(getattr(settings, template.format(property='NODE_NUM'))),
            'CLUSTER': getattr(settings, template.format(property='CLUSTER'))
        }

    settings = build_db_settings(urn, "DAILY_EST")

    sql = '''select app_id from plproxy.execute_select_nestloop($$ 
                select app_id from (
                    select distinct app_id,store_id ,device_id, date, est_organic_download_share
                        from da.app_da_daily_estimate_1001 where date='{}' and store_id=10 )
                        as prod $$) tbl 
              (app_id bigint ) order by app_id  ;
            '''.format(
            date
    )

    runner = LocalSqlRunner(settings)
    rows, _, columns = runner.select_return_columns(sql)
    return [Row(app_id=r[0]) for r in rows]


def citus_row(date):
    def get_data_in_citus(date):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select app_id from store.store_est_fact_v6 where date = '{}' and country_code='US' and est_paid_download !=0  and device_code='android-all' order by app_id".format(
            date)
        db_data = query(citus_dsn_, sql)
        return db_data

    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result

    result = get_data_in_citus(date)
    return [Row(app_id=r[0]) for r in result]


plproxy_result = plproxy_row(date)
citus_result = citus_row(date)

df_plproxy = spark.createDataFrame(plproxy_result).cache()
df_plproxy.createOrReplaceTempView("temp_plproxy")

citus_df = spark.createDataFrame(citus_result).cache()
citus_df.createOrReplaceTempView("temp_citus")
spark.sql("select * from temp_citus except select * from temp_plproxy ").show()
diff_df_list = spark.sql("select * from temp_plproxy except select * from temp_citus ").collect()
print 'not equal count', len(diff_df_list)
diff_list = [str(x[0]) for x in diff_df_list]
df = spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date={}".format(date)).cache()
df.createOrReplaceTempView("temp")
spark.sql("select * from temp where id in ({}) and store_id=10 and feed in (0,1)".format(",".join(diff_list))).show()


In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
select distinct app_id,store_id,device_id, date from plproxy.execute_select_nestloop(\$proxy\$ 
select  distinct app_id,store_id ,device_id, date
    from da.app_da_daily_estimate_1001 where date='2020-01-01' and app_id = 20600003289877 and store_id=10
     limit 5 \$proxy\$) tbl 
      (app_id BIGINT, store_id INT, device_id SMALLINT, date Date )  ;

EOF


# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# select sum(cnt) as sum from plproxy.execute_select_nestloop(\$proxy\$ 
# select count(1) as cnt from (
# select distinct app_id,store_id ,device_id, date, est_organic_download_share
#     from da.app_da_daily_estimate_1001 where ( date  between '2020-01-01' and '2020-01-01') and store_id=5 )
#     as prod
#  \$proxy\$) tbl 
#       (cnt bigint )  ;

# EOF

# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# select app_id from plproxy.execute_select_nestloop(\$proxy\$ 
# select app_id from (
# select distinct app_id,store_id ,device_id, date, est_organic_download_share
#     from da.app_da_daily_estimate_1001 where ( date  between '2020-01-01' and '2020-01-01') and store_id=5 )
#     as prod
#  \$proxy\$) tbl 
#       (app_id bigint ) order by app_id  ;

# EOF



In [0]:

import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
from aadatapipelinecore.core.utils.spark import eject_all_caches

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

# a week data
start = "2018-03-28"
end = "2018-04-01"

real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(dates.pop())
        sar_list.append({real_date1 + datetime.timedelta(days):temp})

test_path=list()
# for x in sar_list:
#     for key,item in x.items():
#         test_path.append(
#             (
#                 ["s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/{}/*/".format(key)] , 
#                 ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v2/fact/granularity=daily/date={}".format(i) for i in item], 
#                 ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(i) for i in item ]
#             )
#         )
        
for x in sar_list:
    for key,item in x.items():
        test_path.append(
            (
                ["s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/{}/*/".format(key)] , 
                [i.strftime("%Y-%m-%d") for i in item], 
                ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(i) for i in item ]
            )
        )
print test_path



class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()



# [(['s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/2020-01-11/*/'], ['2020-01-11', '2020-01-10', '2020-01-09', '2020-01-08', '2020-01-07', '2020-01-06', '2020-01-05'], ['s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-01-11', 's3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-01-10', 's3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-01-09', 's3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-01-08', 's3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-01-07', 's3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-01-06', 's3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-01-05'])]


def test_download_attribution(test_data):
    print test_data
    spark.read.option("basePath",
                      "s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/").parquet(
        test_data[0][0]).createOrReplaceTempView(
        "download_attribution")

    # spark.read.option("basePath",
    #                   "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily").parquet(
    #     "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={%s}" % ",".join(test_path[0][1])).createOrReplaceTempView(
    #     "store_unified")

    # spark.read.option("basePath",
    #                   "s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/2016-09-03/").parquet(
    #     test_data[0][0]).createOrReplaceTempView(
    #     "download_attribution")
    
    print test_data[1]
    spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v3/fact/").where("granularity='daily' and  date in ('{}') ".format("','".join(
        test_data[1]))).createOrReplaceTempView("test_unified_download_attribution")

    sql_text = """
    
    WITH download_attribution AS (
        select *, CAST(est_non_organic_download_share as decimal(36,20)) as new_est_non_organic_download_share from download_attribution
    );

    WITH download_attribution_1 AS (
    
     select device_code, country_code, granularity, date, product_id, new_est_non_organic_download_share, 
        case when device_code='android-phone' THEN 'android-all' 
             when device_code='ios-phone' THEN 'ios-tablet' END AS new_device_code from download_attribution
    );

    WITH download_attribution_2
    AS (
        select distinct * from ( select new_device_code as device_code, country_code, granularity, date, product_id, new_est_non_organic_download_share as est_non_organic_download_share from download_attribution_1 
        union all
        select device_code, country_code, granularity, date, product_id, est_non_organic_download_share from download_attribution ) as t1 where device_code!='android-phone'

    );
    
    WITH union_data AS (
    select *, store_unified.device_code as unified_device_code , store_unified.country_code as unified_country_code
    from store_unified full outer join download_attribution_2
    on store_unified.device_code=download_attribution_2.device_code and
    store_unified.country_code=UPPER(download_attribution_2.country_code) and
    store_unified.app_id=download_attribution_2.product_id
    where est_non_organic_download_share is not null
    );
    
    
    WITH calculate_data_prepare AS (
    select app_id, coalesce(free_app_download, 0) as free_app_download,  coalesce(paid_app_download, 0 )  as paid_app_download, revenue, unified_device_code, unified_country_code, est_non_organic_download_share from union_data where not (free_app_download is null and paid_app_download is null) 
    );
    
    

    
    WITH caculate_data AS (
    select app_id,free_app_download,paid_app_download,revenue, unified_device_code, unified_country_code, est_non_organic_download_share from calculate_data_prepare
    );
    
    
    
    WITH compare_data_raw AS (
    select app_id,free_app_download, paid_app_download, unified_device_code as device_code, unified_country_code as country_code, (1 - est_non_organic_download_share) AS organic_download_share from caculate_data
    );
    
    
    WITH compare_data_unified AS (
    select app_id,coalesce(free_app_download, 0 ) as free_app_download,  coalesce(paid_app_download, 0 ) as paid_app_download,  device_code,  country_code, organic_download_share from test_unified_download_attribution
    );
    
    

    """
    
    # store_unified , rank_unified
    namespace = "aa.store.market-size.v1"
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
            {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": test_data[2]
            }
            # ,
            # {
            #     "data_encoding": "parquet",
            #     "compression": "gzip",
            #     "name": "test_unified_download_attribution",
            #     "path":test_data[1]
            # }
    
        ]
    }
    
    run(spark, ingest_msg, sql_text)
    spark.sql("select * from compare_data_raw where app_id = 284882215 and country_code='US'").show()
    # spark.sql("select * from compare_data_raw where app_id is not null except all select * from compare_data_unified where app_id is not null").show()
    # spark.sql("select * from compare_data_unified except all select * from compare_data_raw").show()
    # count_1 = spark.sql("select count(*) from compare_data_raw where app_id is not null").take(1)
    # count_2 = spark.sql("select count(*) from compare_data_unified ").take(1)
    # if count_1[0][0] != count_2[0][0]:
    #     print 'failed!!!!!!!!!!!!!'


    eject_all_caches(spark)


sc.parallelize(map(test_download_attribution, test_path), 1)

In [0]:

spark.sql("select * from compare_data_raw").show()
spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v3/fact/").where("granularity='daily' and  date ='{}' ".format("2018-05-10")).createOrReplaceTempView("test_unified_download_attribution")

spark.sql("select * from test_unified_download_attribution where app_id = 284882215 and country_code='US'").show()

In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v3/fact/granularity=daily/date=2020-05-30/ --recursive | sort -n 
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-05-30/ --recursive  | sort -n 


In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;

-- est_free_app_download + est_paid_app_download = organic_download + paid_download
select distinct date from store_est_fact_v6 where  country_code='US' and est_paid_download !=0  and device_code='ios-phone' and date between '2016-08-26' and '2020-05-30' order by date ;
select distinct date from store_est_category_fact_v7 where  country_code='US' and est_paid_download !=0  and device_code='ios-phone' and date between '2016-08-26' and '2020-05-30' order by date ;

-- select * from store_est_fact_v6 where date = '2020-03-05' and country_code='AU' and device_code='ios-phone' and est_paid_download is not null order by est_paid_download desc limit 5
-- select * from store_est_category_fact_v7 where date = '2020-01-01' and country_code='US'  and device_code='android-all' and app_id = 20600006409047;

-- select (sum(est_free_app_download) + sum(est_paid_app_download)) as sum_free_paid, (sum(est_paid_download) + sum(est_organic_download)) as sum_orgainc_paid from store_est_fact_v6 where date between '2019-08-29' and '2020-05-30' limit 5 ;
-- select * from store_est_category_fact_v7 where app_id=446366839 and country_code='AU'  and device_code='ios-phone' and date between '2017-06-05' and '2017-06-05' limit 5 ;


EOF

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;

-- Sample test - compare with UI
-- select * from store_est_t_w_fact_v6 where device_code='ios-tablet' and country_code='GB' and date ='2019-06-15' and est_paid_download!=0 limit 3;
-- select * from store_est_t_m_fact_v6 where device_code='android-all' and country_code='JP' and date ='2019-07-31'  and est_paid_download!=0  limit 3;
--select * from store_est_t_y_fact_v6 where device_code='ios-phone' and country_code='WW' and date ='2019-12-31' and est_paid_download!=0 limit 3;
--select * from store_est_t_y_fact_v6 where device_code='android-all' and country_code='WW' and date ='2019-12-31' and est_paid_download!=0 limit 3;

-- select * from store_est_category_t_q_fact_v7 limit 3;
-- select * from store_est_category_t_y_fact_v7 limit 3;

-- Completeness test, date dimension
--select count(distinct date) from store_est_category_t_w_fact_v7 where country_code='US' and est_paid_download !=0 and est_paid_download is not null and device_code='ios-phone' and date between '2016-08-26' and '2020-05-30'  ;
--select distinct date from store_est_t_m_fact_v6 where country_code='US' and est_paid_download !=0 and est_paid_download is not null and device_code='ios-phone' and date between '2016-08-26' and '2020-05-30' order by date 
-- select distinct date from store_est_category_t_m_fact_v7 where country_code='US' and est_paid_download !=0 and est_paid_download is not null and device_code='ios-phone' and date between '2016-08-26' and '2020-05-30' order by date;


-- select * from store_est_category_fact_v7 where date = '2020-01-01' and country_code='US'  and device_code='android-all' and app_id = 20600006409047;

-- select (sum(est_free_app_download) + sum(est_paid_app_download)) as sum_free_paid, (sum(est_paid_download) + sum(est_organic_download)) as sum_orgainc_paid from store_est_fact_v6 where date between '2019-08-29' and '2020-05-30' limit 5 ;


-- select * from store_est_category_fact_v7 where app_id=446366839 and country_code='AU'  and device_code='ios-phone' and date between '2017-06-05' and '2017-06-05' limit 5 ;



-- select sum(est_organic_download), sum(est_paid_download) from store_est_fact_v6 where date between '2019-10-01' and '2019-10-31' and device_code='ios-phone' and country_code='US' and app_id=1225683141 limit 3;
-- select * from store_est_t_m_fact_v6 where device_code='ios-phone' and country_code='US' and date ='2019-10-31' and app_id=1225683141 limit 3;
--select sum(est_free_app_download) as sum_download, sum(est_revenue) as sum_revenue, sum(est_organic_download) as sum_organic_download, sum(est_paid_download) as sum_paid_download from store_est_fact_v6 where date between '2019-10-01' and '2019-12-31' and --device_code='ios-phone' and country_code='US' and app_id=1225683141 limit 3;
--select * from store_est_t_q_fact_v6 where device_code='ios-phone' and country_code='US' and date ='2019-12-31' and app_id=1225683141 limit 3;



--select sum(est_free_app_download) as sum_download, sum(est_revenue) as sum_revenue, sum(est_organic_download) as sum_organic_download, sum(est_paid_download) as sum_paid_download from store_est_fact_v6 where date between '2018-01-01' and '2018-12-31' and -- device_code='ios-phone' and country_code='US' and app_id=1225683141 limit 3;

select * from store_est_t_w_fact_v1 where country_code='US' and est_paid_download !=0 and est_paid_download is not null and device_code='ios-phone' and date between '2016-08-26' and '2020-05-30' limit 5;
select * from store_est_t_m_fact_v1 where country_code='US' and est_paid_download !=0 and est_paid_download is not null and device_code='ios-phone' and date between '2016-08-26' and '2020-01-30' limit 5;



EOF

In [0]:

import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta


# value = ['2016-12-31','2015-06-30', '2012-09-30', '2013-03-31', '2018-03-31'] 
# value = ['2012-12-01','2016-10-22', '2013-05-11', '2013-06-08', '2015-05-02', '2012-07-14', '2013-03-30', '2017-02-04' ,'2018-01-20']
# value = ['2012-12-31','2016-10-31', '2013-05-31', '2013-06-30', '2015-05-31', '2012-07-31', '2013-03-31', '2017-02-28' ,'2018-01-31']

granularity = 'quarterly'

start = "2019-01-01"
end = "2019-04-01"

def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_date_range(date_list, granularity):
    result = []
    start = datetime.datetime.strptime(date_list[0], '%Y-%m-%d')
    end = datetime.datetime.strptime(date_list[1], '%Y-%m-%d')
    if granularity == 'weekly':
        while start <= end:
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(weeks=1)
    elif granularity == 'monthly':
        while start <= end:
            start = last_day_of_month(start)
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(months=1)
    elif granularity == 'quarterly':
        while start <= end:
            start += relativedelta(months=2)
            start = last_day_of_month(start)
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            # start += relativedelta(months=3)
    elif granularity == 'yearly':
        while start <= end:
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(months=12)
    # print result
    return result

agg_date = get_date_range([ start,end ], granularity)

agg_date.sort()
print agg_date
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
print date_range.days
for days in xrange(date_range.days):
    dates.append(str(real_date1 + datetime.timedelta(days)))
    

temp_list = list()
i = 0 
for x in dates:
    temp_list.append(x)
    
    if i < len(agg_date) and x == agg_date[i]  :
        print x
        sar_list.append((agg_date[i], temp_list))
        i = i+1
        temp_list=[]

print sar_list

In [0]:

import datetime
import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F


granularity = 'quarterly'

store_db = "store_est_fact_v1"

granularity_table_dict = {
                        'weekly': ['store.app-est.v3', store_db],
                         'monthly': ['store.app-est.v3', store_db],
                         'quarterly' : ['store.app-est-pre-aggr.v3', store_db],
                         'yearly' : ['store.app-est-pre-aggr.v3', store_db]
                        }



download_attribution_bucket='store.download-attribution-pre-aggr.v3'


def citus_row(date, granularity):
    def get_data_in_citus(date):
        print 'get data in citus ', date
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = '''SELECT  
                    cast(sum(est_free_app_download) as bigint) as est_free_app_download, 
                    cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, 
                    cast(sum(est_revenue) as bigint) as est_revenue, 
                    cast(sum(est_organic_download) as bigint) as est_organic_download ,  
                    cast(sum(est_paid_download) as bigint) as est_paid_download
                FROM store.{}
                WHERE date in ('{}') '''.format(granularity_table_dict[granularity][1], "','".join(date[1:]))
        print sql
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(date)
    return [Row(sum_est_free_app_download=r[0], sum_est_paid_app_download=r[1], sum_est_revenue=r[2], sum_est_organic_download=r[3], sum_est_paid_download=r[4] ) for r in result]


for d in sar_list:
    print d[0]

    daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/{}/fact/".format(download_attribution_bucket)).where("granularity='%s' and date in ('%s')" % (granularity, d[0]))
    daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")
    
    est_weekly_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/{}/fact/".format(granularity_table_dict[granularity][0])).where("granularity='{}' and date in ('{}')".format(granularity,d[0] ))
    est_weekly_df.createOrReplaceTempView("est_weekly_df")


    citus_result = citus_row(d[1], granularity)

    schema = StructType([
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True),
    StructField("sum_est_organic_download", LongType(), True),
    StructField("sum_est_paid_download", LongType(), True)])

    df_3 = spark.createDataFrame(citus_result, schema)
    df_3.createOrReplaceTempView("citus_data")

    
    # compare est paid download by DB and download attribution
    spark.sql("select sum(est_paid_download) as sum_est_paid_download from download_attribution_weekly_unified except all select sum_est_paid_download from citus_data").show()
    spark.sql("select sum_est_paid_download from citus_data except all select sum(est_paid_download) as sum_est_paid_download from download_attribution_weekly_unified").show()

    # compare est weekly data with DB metric sum(free_app_download), sum(paid_app_download), sum(revenue) 
    spark.sql("select sum(free_app_download), sum(paid_app_download), sum(revenue) from est_weekly_df except all select sum_est_free_app_download, sum_est_paid_app_download, sum_est_revenue from citus_data ").show()
    spark.sql("select sum_est_free_app_download, sum_est_paid_app_download, sum_est_revenue from citus_data except all select sum(free_app_download), sum(paid_app_download), sum(revenue) from est_weekly_df ").show()


    # compare organic + paid = free_app + paid_app in DB
    list_result =  spark.sql("select sum_est_free_app_download + sum_est_paid_app_download,  sum_est_organic_download + sum_est_paid_download from citus_data ").collect()
    if list_result[0][0] != list_result[0][1]:
        print 'compare compare organic + paid = free_app + paid_app failed!! , organic + paid= {}, free_app + paid_app = {}'.format(list_result[0][0], list_result[0][1])
    else:
        print 'pass!'

In [0]:

spark.sql("select sum_est_paid_download from citus_data").show()

In [0]:
%%sh

PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select sum(est_paid_download) from store_est_t_q_fact_v1 where date between '2019-01-01' and '2019-03-31' ;
select sum(est_paid_download) from store_est_fact_v1 where date between '2019-01-01' and '2019-03-31' ;
-- select count(1) from store_est_t_w_fact_v1 where date between '2012-08-12' and '2012-08-18' ;
-- select  count(1) from store_est_category_t_w_fact_v1 where date between '2012-08-12' and '2012-08-18' ;
EOF

In [0]:

import datetime
import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F

start = "2019-01-01"
end = "2019-04-01"



# value = ['2016-12-31','2015-06-30', '2012-09-30', '2013-03-31', '2018-03-31'] 
# value = ['2012-12-01','2016-10-22', '2013-05-11', '2013-06-08', '2015-05-02', '2012-07-14', '2013-03-30', '2017-02-04' ,'2018-01-20']
# value = ['2012-12-31','2016-10-31', '2013-05-31', '2013-06-30', '2015-05-31', '2012-07-31', '2013-03-31', '2017-02-28' ,'2018-01-31']

granularity = 'daily'

real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    # if (real_date1 + datetime.timedelta(days)).weekday() == 5:
    #     temp=list()
    #     while dates:
    #         temp.append(str(dates.pop()))
    #     sar_list.append((str(real_date1 + datetime.timedelta(days)), temp))




def citus_row(date, granularity):
    def get_data_in_citus(date):
        print 'get data in citus ', date
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = '''SELECT  
                    cast(sum(est_free_app_download) as bigint) as est_free_app_download, 
                    cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, 
                    cast(sum(est_revenue) as bigint) as est_revenue, 
                    cast(sum(est_organic_download) as bigint) as est_organic_download ,  
                    cast(sum(est_paid_download) as bigint) as est_paid_download
                FROM store.{}
                WHERE date in ('{}') '''.format(granularity_table_dict[granularity][1],date)
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(date)
    return [Row(sum_est_free_app_download=r[0], sum_est_paid_app_download=r[1], sum_est_revenue=r[2], sum_est_organic_download=r[3], sum_est_paid_download=r[4] ) for r in result]

store_db = "store_est_fact_v1"
download_attribution_bucket = "store.download-attribution-load.v3"
granularity_table_dict = {
                        'daily' : ['store.app-est-load.v3', store_db],
                        'weekly': ['store.app-est.v3', store_db],
                         'monthly': ['store.app-est.v3', store_db],
                         'quarterly' : ['store.app-est-pre-aggr.v3', store_db],
                         'yearly' : ['store.app-est-pre-aggr.v3', store_db]
                        }


for d in dates:

    daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/{}/fact/".format(download_attribution_bucket)).where("granularity='%s' and date in ('%s')" % (granularity, d))
    daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")
    
    est_weekly_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/{}/fact/".format(granularity_table_dict[granularity][0])).where("granularity='{}' and date in ('{}')".format(granularity,d ))
    est_weekly_df.createOrReplaceTempView("est_weekly_df")


    citus_result = citus_row(d, 'daily')

    schema = StructType([
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True),
    StructField("sum_est_organic_download", LongType(), True),
    StructField("sum_est_paid_download", LongType(), True)])

    df_3 = spark.createDataFrame(citus_result, schema)
    df_3.createOrReplaceTempView("citus_data")

    
    # compare est paid download by DB and download attribution
    spark.sql("select sum(est_paid_download) as sum_est_paid_download from download_attribution_weekly_unified except all select sum_est_paid_download from citus_data").show()
    spark.sql("select sum_est_paid_download from citus_data except all select sum(est_paid_download) as sum_est_paid_download from download_attribution_weekly_unified").show()

    # compare est weekly data with DB metric sum(free_app_download), sum(paid_app_download), sum(revenue) 
    spark.sql("select sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue) from est_weekly_df except all select sum_est_free_app_download, sum_est_paid_app_download, sum_est_revenue from citus_data ").show()
    spark.sql("select sum_est_free_app_download, sum_est_paid_app_download, sum_est_revenue from citus_data except all select sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue) from est_weekly_df ").show()


    # compare organic + paid = free_app + paid_app in DB
    list_result =  spark.sql("select sum_est_free_app_download + sum_est_paid_app_download,  sum_est_organic_download + sum_est_paid_download from citus_data ").collect()
    if list_result[0][0] != list_result[0][1]:
        print 'compare compare organic + paid = free_app + paid_app failed!! , organic + paid= {}, free_app + paid_app = {}'.format(list_result[0][0], list_result[0][1])
    else:
        print 'pass!'

In [0]:

import datetime
import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F

start = "2016-08-28"
end = "2020-03-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(str(dates.pop()))
        sar_list.append([str(real_date1 + datetime.timedelta(days)),temp])


granularity = 'weekly'
store_db = "store_est_fact_v1"

granularity_table_dict = {
                        'weekly': ['store.app-est.v3', store_db],
                         'monthly': ['store.app-est.v3', store_db],
                         'quarterly' : ['store.app-est-pre-aggr.v3', store_db],
                         'yearly' : ['store.app-est-pre-aggr.v3', store_db]
                        }


download_attribution_bucket='store.download-attribution-pre-aggr.v3'


def citus_row(date, granularity):
    def get_data_in_citus(date):
        print 'get data in citus ', date
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = '''SELECT  
                    cast(sum(est_free_app_download) as bigint) as est_free_app_download, 
                    cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, 
                    cast(sum(est_revenue) as bigint) as est_revenue, 
                    cast(sum(est_organic_download) as bigint) as est_organic_download ,  
                    cast(sum(est_paid_download) as bigint) as est_paid_download
                FROM store.{}
                WHERE date between '{}' and '{}' '''.format(granularity_table_dict[granularity][1], date[1][-1], date[1][0])
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(date)
    return [Row(sum_est_free_app_download=r[0], sum_est_paid_app_download=r[1], sum_est_revenue=r[2], sum_est_organic_download=r[3], sum_est_paid_download=r[4] ) for r in result]


for d in sar_list:
    print 'd[0][0] is ', d[0]
    print d
    daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/{}/fact/".format(download_attribution_bucket)).where("granularity='%s' and date in ('%s')" % (granularity, d[0]))
    daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")
    
    est_weekly_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/{}/fact/".format(granularity_table_dict[granularity][0])).where("granularity='{}' and date in ('{}')".format(granularity,d[0] ))
    est_weekly_df.createOrReplaceTempView("est_weekly_df")


    citus_result = citus_row(d, 'weekly')

    schema = StructType([
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True),
    StructField("sum_est_organic_download", LongType(), True),
    StructField("sum_est_paid_download", LongType(), True)])

    df_3 = spark.createDataFrame(citus_result, schema)
    df_3.createOrReplaceTempView("citus_data")

    
    # compare est paid download by DB and download attribution
    spark.sql("select sum(est_paid_download) as sum_est_paid_download from download_attribution_weekly_unified except all select sum_est_paid_download from citus_data").show()
    spark.sql("select sum_est_paid_download from citus_data except all select sum(est_paid_download) as sum_est_paid_download from download_attribution_weekly_unified").show()

    # compare est weekly data with DB metric sum(free_app_download), sum(paid_app_download), sum(revenue) 
    spark.sql("select sum(free_app_download), sum(paid_app_download), sum(revenue) from est_weekly_df except all select sum_est_free_app_download, sum_est_paid_app_download, sum_est_revenue from citus_data ").show()
    spark.sql("select sum_est_free_app_download, sum_est_paid_app_download, sum_est_revenue from citus_data except all select sum(free_app_download), sum(paid_app_download), sum(revenue) from est_weekly_df ").show()


    # compare organic + paid = free_app + paid_app in DB
    list_result =  spark.sql("select sum_est_free_app_download + sum_est_paid_app_download,  sum_est_organic_download + sum_est_paid_download from citus_data ").collect()
    if list_result[0][0] != list_result[0][1]:
        print 'compare compare organic + paid = free_app + paid_app failed!! , organic + paid= {}, free_app + paid_app = {}'.format(list_result[0][0], list_result[0][1])
    else:
        print 'pass!'

In [0]:


import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta


# value = ['2016-12-31','2015-06-30', '2012-09-30', '2013-03-31', '2018-03-31'] 
# value = ['2012-12-01','2016-10-22', '2013-05-11', '2013-06-08', '2015-05-02', '2012-07-14', '2013-03-30', '2017-02-04' ,'2018-01-20']
# value = ['2012-12-31','2016-10-31', '2013-05-31', '2013-06-30', '2015-05-31', '2012-07-31', '2013-03-31', '2017-02-28' ,'2018-01-31']

granularity = 'quarterly'

start = "2011-01-01"
end = "2020-01-01"

def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_date_range(date_list, granularity):
    result = []
    start = datetime.datetime.strptime(date_list[0], '%Y-%m-%d')
    end = datetime.datetime.strptime(date_list[1], '%Y-%m-%d')
    if granularity == 'weekly':
        while start <= end:
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(weeks=1)
    elif granularity == 'monthly':
        while start <= end:
            start = last_day_of_month(start)
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(months=1)
    elif granularity == 'quarterly':
        while start < end:
            if start.month in [1, 4, 7, 10]:
                start += relativedelta(months=2)
            elif start.month in [2, 5, 8, 11]:
                start += relativedelta(months=1)
            else:
                start += relativedelta(months=3)
            start = last_day_of_month(start)
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
    elif granularity == 'yearly':
        while start <= end:
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(months=12)
    # print result
    return result

agg_date = get_date_range([ start,end ], granularity)

agg_date.sort()
print agg_date


real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
print date_range.days
for days in xrange(date_range.days):
    dates.append(str(real_date1 + datetime.timedelta(days)))
    

temp_list = list()
i = 0 
for x in dates:
    temp_list.append(x)
    
    if i < len(agg_date) and x == agg_date[i]  :
        print x
        sar_list.append((agg_date[i], temp_list))
        i = i+1
        temp_list=[]

print sar_list

In [0]:

import datetime
import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F


granularity = 'quarterly'

store_db = "store_est_fact_v1"

granularity_table_dict = {
                        'weekly': ['store_est_t_w_fact_v1', store_db],
                         'monthly': ['store_est_t_m_fact_v1', store_db],
                         'quarterly' : ['store_est_q_w_fact_v1', store_db],
                         'yearly' : ['store_est_t_y_fact_v1', store_db]
                        }





def citus_row(date, granularity):
    def get_data_in_citus(date):
        # print 'get data in citus ', date
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = '''SELECT  
                    cast(sum(est_free_app_download) as bigint) as est_free_app_download, 
                    cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, 
                    cast(sum(est_revenue) as bigint) as est_revenue, 
                    cast(sum(est_organic_download) as bigint) as est_organic_download ,  
                    cast(sum(est_paid_download) as bigint) as est_paid_download
                FROM store.{}
                WHERE date in ('{}') '''.format(granularity_table_dict[granularity][1], "','".join(date))
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(date)
    return [Row(sum_est_free_app_download=r[0], sum_est_paid_app_download=r[1], sum_est_revenue=r[2], sum_est_organic_download=r[3], sum_est_paid_download=r[4] ) for r in result]


for d in sar_list:
    citus_result_daily = citus_row(d[1], granularity)

    schema = StructType([
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True),
    StructField("sum_est_organic_download", LongType(), True),
    StructField("sum_est_paid_download", LongType(), True)])

    df_3 = spark.createDataFrame(citus_result_daily, schema)
    df_3.createOrReplaceTempView("citus_data_daily")


    citus_pre_agg_result = citus_row(d[1], granularity)

    schema = StructType([
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True),
    StructField("sum_est_organic_download", LongType(), True),
    StructField("sum_est_paid_download", LongType(), True)])

    df_3 = spark.createDataFrame(citus_pre_agg_result, schema)
    df_3.createOrReplaceTempView("citus_data_pre_agg")

    
    # compare est paid download by DB and download attribution
    spark.sql("select * from citus_data_daily except all select * from citus_data_pre_agg").show()
    spark.sql("select * from citus_data_pre_agg except all select * from citus_data_daily").show()



    # compare organic + paid = free_app + paid_app in DB
    list_result =  spark.sql("select sum_est_free_app_download + sum_est_paid_app_download,  sum_est_organic_download + sum_est_paid_download from citus_data_pre_agg ").collect()
    if list_result[0][0] != list_result[0][1]:
        print 'compare compare organic + paid = free_app + paid_app failed!! , organic + paid= {}, free_app + paid_app = {}'.format(list_result[0][0], list_result[0][1])
    else:
        print 'pass!'

In [0]:

import datetime

start = "2018-05-06"
end = "2018-05-12"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(str(dates.pop()))
        sar_list.append((str(real_date1 + datetime.timedelta(days)), temp))
print sar_list
for d in sar_list:
    daily_download_attr = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/").where("granularity='daily' and date in ('%s')" % ("','".join(sar_list[0][1])))
    daily_download_attr.createOrReplaceTempView("daily_pre_load_data")
    
    daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/").where("granularity='weekly' and date in ('%s')" % (sar_list[0][0]))
    daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")
    
    

    spark.sql("select app_id, country_code, device_code, est_organic_download, est_paid_download from download_attribution_weekly_unified except all select app_id, country_code, device_code, sum(est_organic_download) as est_organic_download, sum(est_paid_download) as est_paid_download from daily_pre_load_data group by app_id, country_code, device_code").show()

    spark.sql("select app_id, country_code, device_code, sum(est_organic_download) as est_organic_download, sum(est_paid_download) as est_paid_download from daily_pre_load_data group by app_id, country_code, device_code except all select app_id, country_code, device_code, est_organic_download, est_paid_download from download_attribution_weekly_unified").show()



In [0]:

import datetime

start = "2020-02-01"
end = "2020-04-01"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
# sar_list=list()
for days in xrange(date_range.days):
    dates.append(str(real_date1 + datetime.timedelta(days)))
#     if (real_date1 + datetime.timedelta(days)).weekday() == 5:
#         temp=list()
#         while dates:
#             temp.append(str(dates.pop()))
#         sar_list.append((str(real_date1 + datetime.timedelta(days)), temp))
# print sar_list

print dates
print dates[-1]
# sql_where = "granularity='daily' and date in ('%s')" % ("','".join(dates))
sql_where = "granularity='daily' and date between '2019-03-01' and '2019-03-31'"

print sql_where
daily_download_attr = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/").where(sql_where)
daily_download_attr.createOrReplaceTempView("daily_pre_load_data")

daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/").where("granularity='monthly' and date='2019-03-31' ")
daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")
    
    
spark.sql("select app_id, country_code, device_code, est_organic_download, est_paid_download from download_attribution_weekly_unified except all select app_id, country_code, device_code, sum(est_organic_download) as est_organic_download, sum(est_paid_download) as est_paid_download from daily_pre_load_data group by app_id, country_code, device_code").show()

spark.sql("select app_id, country_code, device_code, sum(est_organic_download) as est_organic_download, sum(est_paid_download) as est_paid_download from daily_pre_load_data group by app_id, country_code, device_code except all select app_id, country_code, device_code, est_organic_download, est_paid_download from download_attribution_weekly_unified").show()


spark.sql("select * from download_attribution_weekly_unified").show()

In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/granularity=daily/date=2018-05-10/device_code=ios-phone/


 


In [0]:


spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/granularity=daily/date=2018-05-10/device_code=ios-phone/").show()